In [2]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

In [3]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,178478,Bachelors,13,Never-married,Tech-support,Own-child,White,Female,0,0,40,United-States,<=50K
1,23,State-gov,61743,5th-6th,3,Never-married,Transport-moving,Not-in-family,White,Male,0,0,35,United-States,<=50K
2,46,Private,376789,HS-grad,9,Never-married,Other-service,Not-in-family,White,Male,0,0,15,United-States,<=50K
3,55,?,200235,HS-grad,9,Married-civ-spouse,?,Husband,White,Male,0,0,50,United-States,>50K
4,36,Private,224541,7th-8th,4,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,40,El-Salvador,<=50K


In [4]:
label='class'
train_data[label].describe()

count      39073
unique         2
top        <=50K
freq       29704
Name: class, dtype: object

In [5]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass\"
AutoGluon Version:  0.5.2
Python Version:     3.9.12
Operating System:   Windows
Train Data Rows:    39073
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' <=50K', ' >50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Using Feature Generators to preprocess the data ...
Fitting

In [6]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States


In [7]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.8763435356740711
Evaluations on test data:
{
    "accuracy": 0.8763435356740711,
    "balanced_accuracy": 0.7950062351568354,
    "mcc": 0.6395678748952276,
    "f1": 0.710727969348659,
    "precision": 0.798708288482239,
    "recall": 0.640207075064711
}


Predictions:  
 0        <=50K
1        <=50K
2         >50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object


In [8]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost,0.877162,0.8872,0.061999,0.019343,0.857542,0.061999,0.019343,0.857542,1,True,11
1,WeightedEnsemble_L2,0.876344,0.8912,0.275789,0.105148,5.275351,0.003968,0.002976,0.941904,2,True,14
2,CatBoost,0.874399,0.8824,0.020834,0.009423,12.676426,0.020834,0.009423,12.676426,1,True,7
3,LightGBMLarge,0.873784,0.8856,0.063984,0.024303,1.064912,0.063984,0.024303,1.064912,1,True,13
4,LightGBM,0.873477,0.8824,0.036208,0.018847,0.557504,0.036208,0.018847,0.557504,1,True,4
5,LightGBMXT,0.871430,0.8792,0.066975,0.022816,1.787025,0.066975,0.022816,1.787025,1,True,3
6,RandomForestGini,0.860579,0.8588,0.272791,0.071424,1.702459,0.272791,0.071424,1.702459,1,True,5
7,NeuralNetTorch,0.858942,0.8600,0.138403,0.053568,57.038164,0.138403,0.053568,57.038164,1,True,12
8,RandomForestEntr,0.857918,0.8620,0.286183,0.065970,1.683049,0.286183,0.065970,1.683049,1,True,6
9,NeuralNetFastAI,0.857816,0.8632,0.151265,0.052575,42.674844,0.151265,0.052575,42.674844,1,True,10
